# Financial Analysis
- Compare companies across industries
- Track financial performance over time
- Calculate and analyze financial ratios
- Build custom financial dashboards

In [1]:
from edgar import *

In [2]:
set_identity("rohilpal9763@gmail.com")

# Core objects

1. Company
    - The Company class is the primary interface for working with public companies in EdgarTools. It provides access to company information, SEC filings, and financial data.
2. Filings
3. Filing
    - The Filing class represents a single SEC filing and provides access to its documents, data, and metadata. It serves as the foundation for all filing-related operations in EdgarTools.

4. XBRL

## Company API reference

1. Company object can be fetched from edgar using cik or ticker (case insensitive).
2. Company object can have one or more ticker names. These can be fetched using `company.tickers`
3. Get filings for a company object.
4. Get financials
5. Get quarterly financials
6. Get Facts
7. Get Latest filing


In [3]:
config = {
    "ticker_name": "NVDA",
}

company = Company(cik_or_ticker=config["ticker_name"])

In [ ]:
# Get properties of the class
def list_properties(cls):
    properties = []
    for name in dir(cls):
        # We need to check the class's __dict__ directly to find property objects
        # because dir() also lists methods and other attributes.
        attr = getattr(cls, name, None)
        if isinstance(attr, property):
            properties.append(name)
    return properties

company_meta_props = list_properties(Company)


for prop_name in company_meta_props:
    try:
        # Use getattr to safely access the property value
        prop_value = getattr(company, prop_name)
        print(f"  {prop_name}: {prop_value}")
    except AttributeError:
        # This might happen if a property's getter raises an error
        # or if there's some unexpected issue (less common for basic properties)
        print(f"  {prop_name}: Could not retrieve value (AttributeError)")
    except Exception as e:
        # Catch other potential exceptions during property access
        print(f"  {prop_name}: Error retrieving value - {e}")



In [5]:
from ollama import chat

messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
]

for part in chat('llama3:instruct', messages=messages, stream=True):
  print(part['message']['content'], end='', flush=True)

What a great question! The sky appears blue to our eyes because of a phenomenon called Rayleigh scattering. This is when shorter, blue wavelengths of light are scattered in all directions by tiny molecules of gases in the atmosphere, such as nitrogen and oxygen.

Here's how it works:

1. Sunlight enters Earth's atmosphere and contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. When this light encounters the small molecules of gases in the air, the shorter wavelengths (like blue and violet) are scattered more than the longer wavelengths (like red and orange). This is because the smaller molecules are better at scattering shorter wavelengths.
3. As a result, our eyes see the scattered blue light as the dominant color, giving the sky its blue appearance.

This effect is not unique to Earth; it's also why sunsets often appear reddish-orange. When the sun is lower in the sky, it has to travel through more of the atmosphere to reach us, 

In [11]:
import requests

ticker_to_cik = {}
name_to_cik = {}
cik_to_metadata = {}


"""
Fetches and processes the company ticker and CIK data from the SEC.
This data will be used to verify entities identified by the LLM.
"""
    
SEC_COMPANY_TICKERS_URL = "https://www.sec.gov/files/company_tickers.json"

print("Loading company data from SEC...")

response = requests.get(SEC_COMPANY_TICKERS_URL, headers={"User-Agent": "rohilpal9763@gmail.com"})
response.raise_for_status()
all_companies = response.json()
for company_data in all_companies.values():
    cik_str = str(company_data['cik_str']).zfill(10)
    ticker = company_data['ticker']
    name = company_data['title']
    if cik_str in cik_to_metadata.keys():
        # We are doing this because the same CIK/company can have multiple ticker names 
        cik_to_metadata[cik_str]["ticker"].append(ticker)
    else:
        cik_to_metadata[cik_str] = {}
        cik_to_metadata[cik_str]["name"] = name
        cik_to_metadata[cik_str]["cik"] = cik_str
        cik_to_metadata[cik_str]["ticker"] = [ticker]
    
    ticker_to_cik[ticker.lower()] = cik_str
    name_to_cik[name.lower()] = cik_str
print(f"Successfully loaded data for {len(cik_to_metadata)} companies.")


Loading company data from SEC...
Successfully loaded data for 7869 companies.


In [12]:
print (f"`cik_to_metadata`: {len(cik_to_metadata)}")
print (f"`ticker_to_cik`: {len(ticker_to_cik)}")
print (f"`name_to_cik`: {len(name_to_cik)}")

`cik_to_metadata`: 7869
`ticker_to_cik`: 10090
`name_to_cik`: 7869


### Conclusion

Same company name and cik can have multiple ticker names

In [13]:
for cik, company_info in cik_to_metadata.items():
    if len(set(company_info["ticker"])) > 1:
        print ("--"*30 + "\n")
        print (f"CIK: {cik}")
        print (f"Company info: {company_info}")
        print ("--"*30 + "\n")

------------------------------------------------------------

CIK: 0001652044
Company info: {'name': 'Alphabet Inc.', 'cik': '0001652044', 'ticker': ['GOOGL', 'GOOG']}
------------------------------------------------------------

------------------------------------------------------------

CIK: 0001067983
Company info: {'name': 'BERKSHIRE HATHAWAY INC', 'cik': '0001067983', 'ticker': ['BRK-B', 'BRK-A']}
------------------------------------------------------------

------------------------------------------------------------

CIK: 0000019617
Company info: {'name': 'JPMORGAN CHASE & CO', 'cik': '0000019617', 'ticker': ['JPM', 'JPM-PC', 'JPM-PD', 'JPM-PJ', 'JPM-PK', 'JPM-PL', 'JPM-PM', 'VYLD', 'AMJB']}
------------------------------------------------------------

------------------------------------------------------------

CIK: 0000070858
Company info: {'name': 'BANK OF AMERICA CORP /DE/', 'cik': '0000070858', 'ticker': ['BAC', 'BML-PG', 'BML-PH', 'BAC-PE', 'BML-PL', 'BAC-PB', 'BML-PJ',

In [19]:
name_to_cik

{'nvidia corp': '0001045810',
 'microsoft corp': '0000789019',
 'apple inc.': '0000320193',
 'alphabet inc.': '0001652044',
 'amazon com inc': '0001018724',
 'meta platforms, inc.': '0001326801',
 'broadcom inc.': '0001730168',
 'tesla, inc.': '0001318605',
 'berkshire hathaway inc': '0001067983',
 'jpmorgan chase & co': '0000019617',
 'walmart inc.': '0000104169',
 'visa inc.': '0001403161',
 'oracle corp': '0001341439',
 'eli lilly & co': '0000059478',
 'spdr s&p 500 etf trust': '0000884394',
 'mastercard inc': '0001141391',
 'netflix inc': '0001065280',
 'exxon mobil corp': '0000034088',
 'johnson & johnson': '0000200406',
 'costco wholesale corp /new': '0000909832',
 'home depot, inc.': '0000354950',
 'bank of america corp /de/': '0000070858',
 'palantir technologies inc.': '0001321655',
 'abbvie inc.': '0001551152',
 'procter & gamble co': '0000080424',
 'sap se': '0001000184',
 'chevron corp': '0000093410',
 'asml holding nv': '0000937966',
 'coca cola co': '0000021344',
 'alibab

In [20]:
response = {
    "companies": [
        {
            "type": "cik",
            "value": "0001546538"
        },
        {
            "type": "name",
            "value": "apple inc."
        },
        {
            "type": "ticker",
            "value": "JPM"
        },
        {
            "type": "ticker",
            "value": "AAPL"
        }
    ],
    "financial_metrics": ["revenue", "gross_profit"],
    "time_period": ["last year", "2025", "Q3 2022"]
}

# the financial metrics are already standardised; so we don't need to touch them
# companies -> we need to standardise and retrieve company meta info

# If type == 'cik', then retrieve info from cik_to_metadata
# If type == 'name', then get cik from name_to_cik, then retrieve info from cik_to_metadata
# If type == 'ticker', then get cik from ticker_to_cik, then retrieve info from cik_to_metadata

extracted_company_entities = response.get("companies")
companies = []

for ent in extracted_company_entities:
    if ent["type"] == "cik":
        cik = ent["value"]
        companies.append(
            cik_to_metadata.get(cik, {})
        )
    elif ent["type"] == "name":
        # closest match of the name -> compare all keys with ent['value]
        match_value = ent["value"]
        cik = name_to_cik.get(match_value.lower(), None)
        companies.append(
            cik_to_metadata.get(cik, {})
        )
    elif ent["type"] == "ticker":
        match_value = ent["value"]
        cik = ticker_to_cik.get(match_value.lower(), None)
        companies.append(
            cik_to_metadata.get(cik, {})
        )




In [21]:
companies

[{'name': 'BTS Group Holdings Public Co Limited/ADR',
  'cik': '0001546538',
  'ticker': ['BTGRF', 'BTLWF']},
 {'name': 'Apple Inc.', 'cik': '0000320193', 'ticker': ['AAPL']},
 {'name': 'JPMORGAN CHASE & CO',
  'cik': '0000019617',
  'ticker': ['JPM',
   'JPM-PC',
   'JPM-PD',
   'JPM-PJ',
   'JPM-PK',
   'JPM-PL',
   'JPM-PM',
   'VYLD',
   'AMJB']},
 {'name': 'Apple Inc.', 'cik': '0000320193', 'ticker': ['AAPL']}]